<a href="https://colab.research.google.com/github/Preethikuppuri/Preethikuppuri/blob/main/resumeranker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy scikit-learn transformers streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 860.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.8 MB/s eta 0:00:00


In [7]:
from google.colab import files
files.upload()  # Select and upload kaggle.json again



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"preethika14","key":"cb3770693da77285522f35bf3f2e7114"}'}

In [8]:
import os
print(os.listdir())  # Should show 'kaggle.json'


['.config', 'kaggle.json', 'sample_data']


In [9]:
!kaggle datasets download -d gauravduttakiit/resume-dataset
!unzip resume-dataset.zip


Dataset URL: https://www.kaggle.com/datasets/gauravduttakiit/resume-dataset
License(s): CC0-1.0
  0% 0.00/383k [00:00<?, ?B/s]
100% 383k/383k [00:00<00:00, 47.7MB/s]
Archive:  resume-dataset.zip
  inflating: UpdatedResumeDataSet.csv  


In [10]:
import pandas as pd

# Load the dataset
df = pd.read_csv("UpdatedResumeDataSet.csv")

# Display the first 5 rows
df.head()


,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [11]:
# Install dependencies (if not installed)
!pip install pandas numpy scikit-learn nltk tensorflow keras

# Import libraries
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Download stopwords for text processing
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
# Load the dataset
df = pd.read_csv("UpdatedResumeDataSet.csv")

# Show first 5 rows
df.head()


,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [13]:
# Define a function to clean text data
def clean_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text

# Apply cleaning function to the 'Resume' column
df['cleaned_resume'] = df['Resume'].apply(clean_text)

# Show cleaned data
df[['Resume', 'cleaned_resume']].head()


,Resume,cleaned_resume
0,Skills * Programming Languages: Python (pandas...,skills programming languages python pandas num...
1,Education Details \r\nMay 2013 to May 2017 B.E...,education details may 2013 may 2017 b e uit rg...
2,"Areas of Interest Deep Learning, Control Syste...",areas interest deep learning control system de...
3,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skills â r â python â sap hana â tableau â sap...
4,"Education Details \r\n MCA YMCAUST, Faridab...",education details mca ymcaust faridabad haryan...


In [14]:
# Convert job categories to numbers
df['Category'] = df['Category'].astype('category').cat.codes

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)  # Limit to 5000 words
X = tfidf.fit_transform(df['cleaned_resume']).toarray()  # Convert to numerical format
y = df['Category'].values  # Labels

# Split data into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
# Define Neural Network Model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(df['Category'].unique()), activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.1582 - loss: 3.1883 - val_accuracy: 0.7409 - val_loss: 3.0308
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.5464 - loss: 2.9406 - val_accuracy: 0.7461 - val_loss: 2.5209
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.6440 - loss: 2.3756 - val_accuracy: 0.8290 - val_loss: 1.7536
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.7294 - loss: 1.6343 - val_accuracy: 0.9689 - val_loss: 1.0753
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.8359 - loss: 1.1162 - val_accuracy: 0.9948 - val_loss: 0.6600
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9054 - loss: 0.7834 - val_accuracy: 0.9948 - val_loss: 0.3991
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9456 - loss: 0.5352 - val_accuracy: 0.9948 - val_loss: 0.2395
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9726 - loss: 0.3661 - val_accuracy: 0.9948 - v

In [16]:
# Evaluate model performance
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9973 - loss: 0.0667 
Test Accuracy: 99.48%


In [17]:
def predict_resume_category(resume_text):
    cleaned_text = clean_text(resume_text)
    vectorized_text = tfidf.transform([cleaned_text]).toarray()
    prediction = model.predict(vectorized_text)
    predicted_category = df['Category'].astype('category').cat.categories[np.argmax(prediction)]
    return predicted_category

# Example resume input
sample_resume = "Experienced software engineer with expertise in Python, Machine Learning, and AI."
print("Predicted Job Category:", predict_resume_category(sample_resume))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicted Job Category: 6


In [28]:
!ngrok authtoken 2uRWffLqmn2XlT1HXkjkpfBylVO_31RTG6n4XGzhjWHR5bG6M
from pyngrok import ngrok
public_url = ngrok.connect(8501, "http")
print("Public URL:", public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://a83b-34-74-232-53.ngrok-free.app" -> "http://localhost:8501"
